In [85]:
from dataclasses import dataclass
import random
from typing import Union, Callable
from numbers import Number
import math
import numpy as np

## Объявление классов
### Первый класс - функция
#### Инициализируется с двумя параметрами:
- функция для дифференцирования
- число переменных функции (Задание 1.5)

#### Содержит три метода:
- Метод для дифференцирования
- Метод для генерации функции (Задание 1.4)
- Метод для численного решения (Задание 1.3)

In [86]:
class Func:
    def __init__(self,gen_func,number_of_variables):
        if(gen_func==True):

            self.func_str=self.gen_f(number_of_variables)
            self.fun=eval(self.func_str)
        else:
            self.fun=eval(gen_func)
        self.num=number_of_variables
        self.answer=[]
        self.x=[]
    def new_func(self,gen_func,number_of_variables):
        if(gen_func):
            self.func_str=self.gen_f(number_of_variables)
            self.fun=eval(self.func_str)
        else:
            self.fun=eval(gen_func)
        self.x.clear()
        self.answer.clear()
        self.num=number_of_variables
    def get_solution(self,*args):
        copy=[]
        self.answer.clear()
        self.x.clear()
        for i in args[0]:
            self.x.append(float(i))
        for i in range(self.num):
            copy.clear()
            for j in range(len(args[0])):
                if j!=i:
                    copy.append(args[0][j])
                else:
                    copy.append(Dual(args[0][j],1.0))
            self.answer.append(self.fun(copy).d)
        return self.answer

    def gen_f(self,num):
        lenn=15
        strr="lambda *x: "
        oper=['+','-','/','*','**']
        varib = []
        numbers=['1','2','3','4','5','6','7','8','9','10']
        flag=0

        for i in range(num):
            varib.append("x[0]["+str(i)+"]")
        funcs = ['log(','sin(','cos(','exp(']
        red=[]
        ins=sum([1 if x in red else 0 for x in varib])
        while((len(red)<=lenn and len(varib)!=ins) or len(varib)!=ins):

            scob=random.randint(1,5)
            if len(red)==0:
                n=random.randint(1,len(funcs)+num+5)
                if n-num<=0:
                    red.append(varib[n-1])
                elif n-num -len(funcs)<=0:
                    red.append(funcs[n-num-1])
                    flag+=1
                elif n-num-len(funcs)==-1:
                    red.append(oper[1])
                elif n-num-len(funcs)==-2:
                    red.append('(')
                    flag+=1
                else:
                    red.append(str(random.randint(1,3)))
            elif (scob==1) and (red[-1] in varib)!=True and (red[-1] in numbers)!=True and red[-1]!=')' and flag==0:
                red.append('(')
                flag+=1
            elif (scob==1) and (red[-1] in funcs)!=True and (red[-1] in oper)!=True and red[-1]!='(' and flag>0:
                red.append(')')
                flag-=1
            else:
                if (red[-1] in oper):
                    n=random.randint(1,len(funcs)+num+5)
                    if n-num<=0:
                        red.append(varib[n-1])
                    elif n-num -len(funcs)<=0:
                        red.append(funcs[n-num-1])
                        flag+=1
                    else:
                        red.append(str(random.randint(1,3)))
                elif (red[-1] in varib) or (red[-1] in numbers):
                    n=random.randint(1,len(oper))
                    red.append(oper[n-1])
                elif (red[-1] in funcs):
                    n=random.randint(1,num+5)
                    if n-num<=0:
                        red.append(varib[n-1])
                    elif n-num ==-1:
                        red.append(oper[1])
                    else:
                        red.append(str(random.randint(1,3)))
                elif red[-1]=='(':
                    n=random.randint(1,len(funcs)+num+5)
                    if n-num<=0:
                        red.append(varib[n-1])
                    elif n-num -len(funcs)<=0:
                        red.append(funcs[n-num-1])
                        flag+=1
                    elif n-num-len(funcs)==-1:
                        red.append(oper[1])
                    elif n-num-len(funcs)==-2:
                        red.append('(')
                        flag+=1
                    else:
                        red.append(str(random.randint(1,3)))
                elif red[-1]==')':
                    n=random.randint(1,len(oper))
                    red.append(oper[n-1])
            ins=sum([1 if x in red else 0 for x in varib])
        if red[-1] in oper or red[-1] in funcs or red[-1]=='(':
            n=random.randint(1,num+5)
            if n-num<=0:
                red.append(varib[n-1])
            else:
                red.append(str(random.randint(1,3)))
        while flag>0:
            red.append(')')
            flag-=1
        for i in red:
            strr+=i
        return strr

    def check(self):
        eps=0.0000000001
        answer=[]
        for i in range(len(self.x)):
            x_right=self.x.copy()
            x_left=self.x.copy()
            x_right[i]=x_right[i]+eps
            x_left[i]=x_left[i]-eps
            answer.append((self.fun(x_right)-self.fun(x_left))/(2*eps))
        return answer

### Второй класс - Dual
#### Содержит два элемента:
- value - значение функции
- d - значение дифференциала

#### Содержит перегрузки следующих операторов:
- сложения
- вычитания
- умножения
- деления
- возведения в степень
- унарный
(Задание 1.1)

In [87]:
@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __pow__(self, other: Union["Dual",Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value**o_value, self.value**o_value*(math.log(self.value)+1)*o_d)
            case Number:
                return Dual(self.value**float(other),float(other)*self.value**float(other-1)*self.d)

    def __rpow__(self, other: Union["Dual",Number]) -> "Dual":
        return Dual(float(other)**self.value,float(other)**self.value*math.log(float(other))*self.d)

    def __neg__(self)->"Dual":
        return Dual(-self.value,-self.d)

    def __sub__(self,other: Union["Dual",Number])->"Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d-o_d)
            case Number():
                return Dual(self.value-float(other),self.d)

    def __rsub__(self,other: Union["Dual",Number])->"Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(o_value - self.value, o_d-self.d)
            case Number():
                return Dual(float(other)-self.value,-self.d)

    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value,o_d):
                return Dual(self.value/o_value,(self.d*o_value - self.value*o_d)/o_value**2)
            case Number():
                return Dual(self.value/float(other),self.d/float(other))

    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value,o_d):
                return Dual(o_value/self.value,(self.value*o_d-self.d*o_value)/self.value**2)
            case Number:
                return Dual(Number/self.value,-Number/self.value**2*self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    __rmul__ = __mul__
    __radd__ = __add__


### Функции (Задание 1.2):
#### Синус для чисел и Dual

In [88]:
def sin(other: Union["Dual", Number]):
    match other:
        case Dual(o_value, o_d):
            return Dual(math.sin(o_value),math.cos(o_value)*o_d)
        case Number():
            return math.sin(float(other))

#### Косинус для чисел и Dual

In [89]:
def cos(other: Union["Dual", Number]):
    match other:
        case Dual(o_value, o_d):
            return Dual(math.cos(o_value),-math.sin(o_value)*o_d)
        case Number():
            return math.cos(float(other))

#### Натуральный логарифм для чисел и Dual

In [90]:
def log(other: Union["Dual", Number]):
    match other:
        case Dual(o_value, o_d):
            return Dual(math.log(o_value),1/o_value*o_d)
        case Number():
            return math.log(float(other))

#### Экспонента для чисел и Dual

In [91]:
def exp(other: Union["Dual", Number]):
    match other:
        case Dual(o_value, o_d):
            return Dual(math.exp(o_value),math.exp(o_value)*o_d)
        case Number():
            return math.exp(float(other))

### Тесты

In [92]:
# func1=Func("lambda *x: exp(x[0][1]*3)*9-x[0][0]",2)
# xses=[5,3]
# print(func1.get_solution(xses))
# print(func1.check())

In [93]:
func=Func(True,3)
p=50
while p>0:
    func.new_func(True,random.randint(1,4))
    xses=[]
    try:
        for i in range(func.num):
            xses.append(random.randint(1,5))
        func.get_solution(xses)

    except:
        func.new_func(True,random.randint(1,4))
    else:
        p-=1
        print(func.func_str)
        print(func.x)
        print(func.answer)
        print(func.check())
        print("-------------------------------------------------------------")
        print("-------------------------------------------------------------")

lambda *x: log(3*log(x[0][3]/1*1)/x[0][3]**2**1**exp(2-x[0][0]-2-x[0][0]+3))-2/2-(cos(x[0][2])**sin(x[0][2]/exp(1+x[0][2]-1)**log(x[0][3])*2**sin(x[0][3]**log(x[0][1]))))
[1.0, 5.0, 5.0, 4.0]
[0.0, -0.011756524015715605, -0.0016187637711513154, -0.3449910265215129]
[0.0, -0.011757261830780408, -0.025590640717609858, -0.3449929231180704]
-------------------------------------------------------------
-------------------------------------------------------------
lambda *x: cos(x[0][2]*sin(x[0][0]+x[0][2]/2)-x[0][3]*x[0][1])
[2.0, 5.0, 4.0, 5.0]
[-0.6395674103723937, -1.2230812593401044, -0.5049098949927877, -1.2230812593401044]
[-0.6395695084648878, -1.2230855217509884, -0.5049100026965903, -1.2230855217509884]
-------------------------------------------------------------
-------------------------------------------------------------
lambda *x: 1/3+2/3-sin(1+x[0][2]-x[0][0])*exp(x[0][2]/3*x[0][1])
[3.0, 3.0, 4.0]
[-22.720847417619233, -66.19460977944074, -26.925109916961333]
[-22.7208474257